# Query EarthScope MT Data Holdings


- based on get_MT_numbers.ipynb originally contributed by Laura Keyson
- modified 20230911
- 


In [1]:
# Imports
from aurora.test_utils.earthscope.helpers import DATA_AVAILABILITY_PATHS

import os
import pandas as pd
import pathlib
import time


In [5]:
# Configure this run

doQuery = True
doSummary = True
doCharts = True
include_restricted = False
if include_restricted:
    public_or_restricted = "restricted"
else:
    public_or_restricted = "public"

In [6]:
out_folder = DATA_AVAILABILITY_PATHS[public_or_restricted]
print(out_folder.absolute())
filebase = "mt_availability"
outfile = out_folder.joinpath(f"{filebase}.txt")

/home/kkappler/.cache/earthscope/data_availability/public


In [7]:
fdsn_URL = "http://service.iris.edu/fdsnws"
channels=['?FE', '?FN', '?FZ','?F1','?F2', '?QE', '?QN', '?QZ', '?Q1', '?Q2']
channels = ','.join(channels)

if doQuery:
    with open(outfile, 'w') as f:
        f.write("net.sta,chan,hours")
        
    sta_URL=f"{fdsn_URL}/station/1/query?cha={channels}&level=channel&format=text&includecomments=true&nodata=204"
    print(sta_URL)

    try:
        network_df=pd.read_csv(sta_URL, sep='|')
    except Exception as e:
        print(f"ERROR with station service {sta_URL}")
        print(f"ERROR: {e}")
        quit()

    network_df
    networks=network_df['#Network '].unique()
    print(f"Identified {len(networks)} unique networks: \n {networks}")
    # grouped = stations.groupby(by=['#Network ', ' Station '])
    # grouped = network_df.groupby(by='#Network ')

    for network in networks:
        print(network)
        netfile = f'{filebase}_{network}.txt'
        netfile = out_folder.joinpath(netfile)
        
        av_URL = f"{fdsn_URL}/availability/1/query?format=text&net={network}&cha={channels}&orderby=nslc_time_quality_samplerate&includerestricted={include_restricted}&nodata=204" 

        print(av_URL)
        try:
            avail = pd.read_csv(av_URL, sep=" ")
        except Exception as e:
            print(f"ERROR with availability service {av_URL} ")
            print(f"ERROR: {e}")
            with open(outfile, 'a') as f:
                f.write(f"\n#ERROR with {network}")
            time.sleep(2)
            continue


        avail.columns = avail.columns.str.strip()
        avail['Latest'] = pd.to_datetime(avail['Latest'], format="%Y-%m-%dT%H:%M:%S.%f") 
        avail['Earliest'] = pd.to_datetime(avail['Earliest'],  format="%Y-%m-%dT%H:%M:%S.%f") 
        avail['Span'] = avail.Latest - avail.Earliest

        avail.to_csv(netfile, index=False)
        
        grouped_chan = avail.groupby(by=['Station','Channel'])
        for name, group in grouped_chan:
            station = name[0]
            channel = name[1]
            total_time = group['Span'].sum()
            with open(outfile, 'a') as f:
                f.write(f"\n{network}.{station},{channel},{'%.2f' % (total_time/pd.Timedelta(hours=1))}")
#                 f.write(f"\n{network}.{station},{channel},{total_time}")
                
        time.sleep(5)

http://service.iris.edu/fdsnws/station/1/query?cha=?FE,?FN,?FZ,?F1,?F2,?QE,?QN,?QZ,?Q1,?Q2&level=channel&format=text&includecomments=true&nodata=204
Identified 21 unique networks: 
 ['1H' '4P' '7I' '8J' '8P' 'AV' 'BK' 'EM' 'II' 'IU' 'N4' 'NV' 'SF' 'SN'
 'US' 'XB' 'XC' 'XX' 'YB' 'Z7' 'ZU']
1H
http://service.iris.edu/fdsnws/availability/1/query?format=text&net=1H&cha=?FE,?FN,?FZ,?F1,?F2,?QE,?QN,?QZ,?Q1,?Q2&orderby=nslc_time_quality_samplerate&includerestricted=False&nodata=204
4P
http://service.iris.edu/fdsnws/availability/1/query?format=text&net=4P&cha=?FE,?FN,?FZ,?F1,?F2,?QE,?QN,?QZ,?Q1,?Q2&orderby=nslc_time_quality_samplerate&includerestricted=False&nodata=204
7I
http://service.iris.edu/fdsnws/availability/1/query?format=text&net=7I&cha=?FE,?FN,?FZ,?F1,?F2,?QE,?QN,?QZ,?Q1,?Q2&orderby=nslc_time_quality_samplerate&includerestricted=False&nodata=204
8J
http://service.iris.edu/fdsnws/availability/1/query?format=text&net=8J&cha=?FE,?FN,?FZ,?F1,?F2,?QE,?QN,?QZ,?Q1,?Q2&orderby=nslc_time_qual

In [8]:
if doSummary:
    
    summaryDF = pd.DataFrame(columns=['Network','Channels', '# Stations','Average Time', 'Total Time'])    
    sumDF = pd.read_csv(outfile)
    sumDF[['net','sta']] = sumDF['net.sta'].str.split('.',expand=True)

    zDF = sumDF[sumDF.chan.str.endswith('Z', na=False)]


    grouped_sum = zDF.groupby(by=['net'])
    
    for name, group in grouped_sum:
        nsta = group.nunique(axis=0)['sta']
        avgTime = group['hours'].mean()
        totalTime = group['hours'].sum()
        chans = sumDF[sumDF['net']==name]['chan'].unique()
        newRow = [name, chans, nsta, avgTime, totalTime]
        
        summaryDF.loc[len(summaryDF.index)] = newRow
        

           
    summaryDF['Average Time'] = summaryDF['Average Time'].round().apply(pd.to_timedelta, unit='H')
    summaryDF['Total Time'] = summaryDF['Total Time'].round().apply(pd.to_timedelta, unit='H')
    
    display(summaryDF)
    

,Network,Channels,# Stations,Average Time,Total Time
0,1H,"[LFE, LFN, LFZ, LQE, LQN]",117,19 days 09:00:00,2267 days 01:00:00
1,4P,"[LFE, LFN, LFZ, LQE, LQN]",978,22 days 21:00:00,22382 days 20:00:00
2,8J,"[LFE, LFN, LFZ, LQE, LQN]",25,79 days 04:00:00,1979 days 01:00:00
3,8P,"[LFE, LFN, LFZ, LQE, LQN]",119,25 days 03:00:00,2989 days 07:00:00
4,AV,"[LFE, LFN, LFZ]",3,262 days 05:00:00,786 days 15:00:00
5,EM,"[LFE, LFN, LFZ, LQE, LQN, VFE, VFN, VFZ, VQE, ...",1728,24 days 05:00:00,41841 days 02:00:00
6,II,"[LFE, LFN, LFZ, BF1, BF2, BFE, BFN, BFZ, LF1, ...",2,4340 days 11:00:00,13021 days 10:00:00
7,IU,"[LF1, LF2, LFZ, UFZ, VFZ, LFE, LFN]",15,2558 days 04:00:00,43489 days 03:00:00
8,N4,"[LF1, LF2, LFZ]",2,1107 days 22:00:00,2215 days 21:00:00
9,NV,"[MF1, MF2, MFZ, MFE, MFN]",2,332 days 07:00:00,664 days 15:00:00


In [17]:
if doCharts:
    from bokeh.plotting import figure, output_file, show, save
    from bokeh.layouts import column, gridplot
    from bokeh.models import ColumnDataSource, HoverTool
    from bokeh.models import Range1d
    from bokeh.io import output_notebook
    
    import os
    import pandas as pd
    
    
    # Get list of all networks that have the availability files
    files = sorted(out_folder.glob(f"{filebase}_*.txt"))
    
    
    
    s = list()
    idx = 0
    for file in files:
        network = file.stem.split("_")[-1]
        print(network)
        
        # Timeline with number of stations depicted by width of line? 
        # Or two lines that have shaded area between?
        thisAvail = pd.read_csv(file,parse_dates=['Earliest','Latest'])
        earliest = min(thisAvail['Earliest'])
        latest = max(thisAvail['Latest'])
        print(earliest, latest)

        
        # create 100 bins for that timeframe(?)
        nbins = 100
        datelist = pd.date_range(earliest, latest, periods=100).tolist()

        numDF = pd.DataFrame(columns=['Date','Lower','Upper'])    

        for date in datelist:
            nsta = len(thisAvail[(thisAvail['Earliest']<=date) & (thisAvail['Latest']>=date)].Station.unique())
            numDF.loc[len(numDF.index)] = [date, -nsta, nsta]
            
        
        output_notebook()
        source = ColumnDataSource(numDF)
        
        if idx > 0:
            tmp_s = figure(width=800, height=100,x_axis_type="datetime",x_range=s[0].x_range)
        else: 
            tmp_s = figure(width=800, height=100,x_axis_type="datetime")
        s.append(tmp_s)
        
        s[idx].varea(x='Date', y1='Upper', y2='Lower', source=source)
        s[idx].y_range = Range1d(-30, 30)
        s[idx].title = network
        s[idx].add_tools(HoverTool(
            tooltips=[
                ( 'count', '@Upper'),
                ( 'time', '@Date'),
            ]))
        idx+=1
    
    p = gridplot(s, ncols=1)
    show(p)
    plot_file_name = out_folder.joinpath("mt_numbers.html")
    print(plot_file_name)
    output_file(plot_file_name)
    save(p)
    


        
        
    
        
        # Histogram of length of each station
        # Histogram of length of each segment? Do I have that info?



1H
2013-04-02 21:24:16+00:00 2018-09-30 18:12:19+00:00


Loading BokehJS ...

4P
2006-08-03 19:39:27+00:00 2017-11-05 18:35:35+00:00


Loading BokehJS ...

7I
2018-03-07 00:05:05.939000+00:00 2018-03-11 23:59:55.939000+00:00


...

Z7
2017-10-21 17:59:05+00:00 2017-11-15 21:49:13.875000+00:00


Loading BokehJS ...

ZU
2020-06-23 21:56:01+00:00 2023-01-13 18:56:57+00:00


Loading BokehJS ...

/home/kkappler/.cache/earthscope/data_availability/public/mt_numbers.html


In [16]:
list(files)


[]